## Grounding DINO 사용 Detection 적용

### 이미지 한장으로 실험해보기

In [1]:
import torch

torch.backends.mps.is_available()

False

In [2]:
import os
HOME = os.getcwd()
print(HOME)
CONFIG_PATH = os.path.join(HOME, "../GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/home/KDT-admin/work/GroundingDINO
/home/KDT-admin/work/GroundingDINO/../GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [3]:
import wget
import os

WEIGHTS_PATH = os.path.join('../GroundingDINO/weights')
# 작동이 잘안됨.. ㅜㅜ

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join("../GroundingDINO/weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

../GroundingDINO/weights/groundingdino_swint_ogc.pth ; exist: True


In [4]:
os.chdir('../GroundingDINO')

In [5]:
os.getcwd()

'/home/KDT-admin/work/GroundingDINO'

#### val set

In [6]:
os.getcwd()

'/home/KDT-admin/work/GroundingDINO'

In [7]:
os.chdir('../')

In [33]:
os.getcwd()

'/home/KDT-admin/work'

In [9]:
import os
import unicodedata

# os.chdir('..')

# 폴더 내의 모든 파일의 이름을 담을 리스트 생성
val_file_names = []

# 폴더 경로 설정
folder_path = "../data/yolo_all/val/image"

# 폴더 내의 모든 파일의 이름을 리스트에 추가
for filename in os.listdir(folder_path):
    # print(filename)
    # 파일의 전체 경로 생성
    file_path = os.path.join(folder_path, filename)
    # 파일인지 확인 후 리스트에 추가
    if os.path.isfile(file_path):
        filename = unicodedata.normalize('NFC', filename) 
        val_file_names.append(filename)
    # val_file_names.append(filename)

#### test set

In [10]:
# 폴더 내의 모든 파일의 이름을 담을 리스트 생성
test_file_names = []

# 폴더 경로 설정
folder_path_test = "../data/yolo_all/test/image"

# 폴더 내의 모든 파일의 이름을 리스트에 추가
for filename in os.listdir(folder_path_test):
    # print(filename)
    # 파일의 전체 경로 생성
    file_path = os.path.join(folder_path_test, filename)
    # 파일인지 확인 후 리스트에 추가
    if os.path.isfile(file_path):
        filename = unicodedata.normalize('NFC', filename)
        test_file_names.append(filename)
    # val_file_names.append(filename)

In [11]:
import unicodedata

#### train set

In [12]:
# 폴더 내의 모든 파일의 이름을 담을 리스트 생성
train_file_names = []

# 폴더 경로 설정
folder_path_train = "../data/yolo_all/train/image"

# 폴더 내의 모든 파일의 이름을 리스트에 추가
for filename in os.listdir(folder_path_train):
    # print(filename)
    # 파일의 전체 경로 생성
    file_path = os.path.join(folder_path_train, filename)
    # 파일인지 확인 후 리스트에 추가
    if os.path.isfile(file_path):
        filename = unicodedata.normalize('NFC', filename) 
        train_file_names.append(filename)
    # val_file_names.append(filename)

In [13]:
len(train_file_names)

21000

### yolo용 txt 다시 생성(yolov8로 디텍션까지 할 경우에만 사용됨)

#### 본격 txt작업 시작

In [66]:
from collections import Counter
import random
import json

# faceExp를 클래스로 매핑하는 딕셔너리
emotion_mapping = {
    "분노": 0,
    "불안": 1,
    "기쁨": 2,
    "중립": 3,
    "당황": 4,
    "슬픔": 5,
    "상처": 6
}
annot_mapping = {0:"annot_A",
                 1:"annot_B",
                 2:"annot_C"}

# JSON 파일 불러오기
def load_json_file(file_path):
    with open(file_path, "r", encoding='utf-8') as f:
        data = json.load(f)
    return data

# x_center = ((box["maxX"] + box["minX"]) / 2) / img_width
# faceExp를 클래스로 매핑하여 YOLO 형식의 레이블로 변환하는 함수
def convert_to_yolo_format_with_class(box, img_width, img_height, emotion):
    x_center = (box["maxX"] + box["minX"]) / (2 * img_width) # 0~1 정규화
    y_center = (box["maxY"] + box["minY"]) / (2 * img_height)
    width = (box["maxX"] - box["minX"]) / img_width # 0~1 정규화
    height = (box["maxY"] - box["minY"]) / img_height # 0~1 정규화
    emotion_class = emotion_mapping[emotion]
    return emotion_class, x_center, y_center, width, height

# JSON 데이터에서 YOLO 형식의 레이블로 변환하여 파일에 저장하는 함수
def save_to_txt_with_class(json_data, output_dir, img_width, img_height, filename):
    all_emotion = [json_data[key]["faceExp"] for key in ["annot_A", "annot_B", "annot_C"]]

    word_counts = Counter(all_emotion)
    # print(word_counts)
    if [word_counts for word_counts in word_counts.values()] == [1, 1, 1]:
        if [word_emotion in json_data['filename'] for word_emotion in word_counts.keys()]:
            # print(json_file[0]['filename'])
            result_list = [word_emotion in json_data['filename'] for word_emotion in word_counts.keys()]
            true_indices = [index for index, value in enumerate(result_list) if value]
            if true_indices == []:
                most_emotion_word = random.choice(all_emotion)
            else:
                most_emotion_word = list(word_counts.keys())[true_indices[0]]
    else:
        most_emotion_word = word_counts.most_common(1)[0][0]
    # print(most_emotion_word)

    annot_index = [i for i, word in enumerate(all_emotion) if word == most_emotion_word]
    # print(annot_index)
    if len(annot_index) != 1:
        random.shuffle(annot_index)
        final_emotion_annot = random.choice(annot_index)

    else:
        final_emotion_annot = annot_index[0]

    key = annot_mapping[final_emotion_annot]
    annots = [(json_data[key]["boxes"], json_data[key]["faceExp"])]
    yolo_labels = [convert_to_yolo_format_with_class(box, img_width, img_height, emotion) for box, emotion in annots]
    filename_without_extension = os.path.splitext(filename)[0]
    output_file = output_dir + "/" + filename_without_extension + ".txt"

    with open(output_file, "w") as f:
        for label in yolo_labels:
            f.write(" ".join(str(coord) for coord in label) + "\n")

# 주어진 JSON 데이터 리스트를 처리하여 YOLO 형식의 레이블 파일로 저장
def process_json_data_with_class(json_list, output_dir, path, file_names):
    for json_data in json_list:
        filename = json_data["filename"]
        image_path = os.path.join(path, filename)
        if not os.path.exists(image_path):
            continue  # 이미지 파일이 존재하지 않으면 다음 반복으로 넘어감
        if unicodedata.normalize('NFC', filename) in file_names:
            img_width, img_height = get_image_size(image_path)
            save_to_txt_with_class(json_data, output_dir, img_width, img_height, filename)

def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size  # (width, height)

In [63]:
os.getcwd()

'/home/KDT-admin/work'

In [68]:
# JSON 데이터 처리(train_set)
output_directory = '../data/yolo_all/val/labels'
val_path = '../data/yolo_all/val/image'

for emotion in emotion_mapping.keys():
    json_list = load_json_file(f"../data/yolo_all/val/label/img_emotion_test_data({emotion}).json")
    process_json_data_with_class(json_list, output_directory, val_path, val_file_names)

In [32]:
os.getcwd()

'/home/KDT-admin/work'

In [34]:
# JSON 데이터 처리(train_set)
output_directory = '../data/yolo_all/test/labels'
test_path = '../data/yolo_all/test/image/'

for emotion in emotion_mapping.keys():
    json_list = load_json_file(f"../data/yolo_all/test/label/img_emotion_test_data({emotion}).json")
    process_json_data_with_class(json_list, output_directory, test_path, test_file_names)

FileNotFoundError: [Errno 2] No such file or directory: '../data/yolo_all/test/label/img_emotion_test_data(분노).json'

In [29]:
os.getcwd()

'/home/KDT-admin/work'

In [30]:
os.pwd()

AttributeError: module 'os' has no attribute 'pwd'

In [69]:
# JSON 데이터 처리(train_set)
output_directory = '../data/yolo_all/train/labels'
train_path = '../data/yolo_all/train/image'

for emotion in emotion_mapping.keys():
    json_list = load_json_file(f"../data/yolo_all/train/label/img_emotion_training_data({emotion}).json")
    process_json_data_with_class(json_list, output_directory, train_path, train_file_names)

### 다시 매핑시켜서 이미지를 yolov8_cls모델에 쓰기위해 이미지 이동

In [54]:
flipped_emotion_mapping = {value: key for key, value in emotion_mapping.items()}

In [55]:
flipped_emotion_mapping

{0: '분노', 1: '불안', 2: '기쁨', 3: '중립', 4: '당황', 5: '슬픔', 6: '상처'}

In [56]:
dict_list = ['anger', 'anxiety', 'happy', 'normal', 'embarrass', 'sad', 'pain']
en_emotion_mapping = emotion_mapping.copy()

for emo, en_emo in zip(emotion_mapping, dict_list):
    en_emotion_mapping[emo] = en_emo

In [57]:
en_emotion_mapping

{'분노': 'anger',
 '불안': 'anxiety',
 '기쁨': 'happy',
 '중립': 'normal',
 '당황': 'embarrass',
 '슬픔': 'sad',
 '상처': 'pain'}

In [58]:
# 폴더 경로
folder_labels_path = '../data/yolo_all/val/image/'

# 폴더 내 모든 파일 및 폴더 목록 가져오기
val_txt_files = os.listdir(folder_path)

# txt 파일만 선택
txt_files = [file for file in val_txt_files if file.endswith('.txt')]
txt_files_names = [file[:-4] for file in val_txt_files if file.endswith('.txt')]

contents_dict = {}
# txt 파일들을 불러와서 출력
for txt_file, txt_file_name in zip(txt_files, txt_files_names):
    txt_file_path = os.path.join(folder_labels_path, txt_file)

    with open(txt_file_path, 'r') as one_txt_file:
        contents = one_txt_file.read()
        contents = contents.split(' ')
        content = flipped_emotion_mapping[int(contents[0])]
        # print(f"파일명: {txt_file}\n내용:\n{contents}\n")
        contents_dict[txt_file_name] = content

In [59]:
contents_dict

{}

In [43]:
folder_image_path = '../data/yolo_all/val/image/'
val_img_files = os.listdir(folder_image_path)

for k,v in contens_dict:
    if v in k:
        val_img_files

NameError: name 'contens_dict' is not defined